In [1]:
#imports
import cv2
import numpy as np
import struct

In [2]:
# Detect Speed of face in 2D video

def calcSpeed2D(prev_faces, faces, num_points, frame):

    '''
    This function calculates a rough equivalent of the instantaneous 2D speed of the detected faces in a video using the velocity 
    formula. For each face, a number of equidistant points (chosen by the user, minimum of 4), are sampled from the bounding box
    and the velocity is calculated from the number of pixels traveled (on average?) by each point of interest. 

    Inputs:
        - prev_faces: list of bounding box coordinates (x, y, w, h) of faces in the most recent previous sampled state
        - faces: list of bounding box coordinates (x, y, w, h) of faces in the current sampled state
        - num_points (int): number of points on bounding box to be selected (minimum of 4, must be a multiple of 4)

    Outputs:
        - None: simply displays the velocity under the bounding box of the face
    '''

    assert (num_points >= 4) and (num_points % 4 == 0), "Number of points sampled must be a positive, non-zero multiple of 4."
    velos = []


    for (xp, yp, wp, hp), (xc, yc, wc, hc) in zip(prev_faces, faces):

        cv2.rectangle(frame, (xc, yc), (xc+wc, yc+hc), (255, 0, 0), 2)

        # sample points of interest on the bounding boxes

        pPois = []
        cPois = []

        bisectors = num_points // 4
        for i in np.arange(bisectors):
            for j in np.arange(bisectors):
                if (i != 0 or i != bisectors-1) or (j != 0 or j != bisectors - 1): # if point would not be on edge of bounding box
                    continue
                else:
                    pPois.append((xp + i*wp/bisectors, yp + j*hp/bisectors))
                    cPois.append((xc + i*wc/bisectors, yc + j*hc/bisectors))
            
        # calculate the differences between the previous and current points of interest
        diff = np.subtract(pPois, cPois)

        # compute the average difference of each point
        avg_diff = np.mean(diff)

        # calculate the speed based on the difference (using 30 fps as the framerate)
        velo = np.linalg.norm(avg_diff) * 30

        # create the output string to go under the bounding box on the video
        out_str = 'Velocity: ' + str(velo) + ' pixels/sec'
        velos.append(velo)

        cv2.putText(frame, out_str, (xc, yc + hc + 20), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 0), 1)
        
    return velos

In [3]:
# Detect Speed of a face in 3D
'''
Idea keep track of previous point and current point 
and to compute the distance between the two points.

p1 (cx, cy, cdepth) p2 (px, py, pdepth)
'''

def calcSpeed3D(frame, dframe, p_depths, faces, odd):
    c_depths = []
    vs = []
    for (x, y, w, h) in faces:
        (depth_value, _) = struct.unpack('hc', dframe[int((x+w/2)/2),int((y+h/2)/2)])
        c_depths.append((int((x+w/2)/2), int((y+h/2)/2), depth_value))
    
    if len(p_depths) == 0 or len(c_depths) == 0:
        return c_depths, vs
    
    for i, ((xp, yp, zp), (xc, yc, zc)) in enumerate(zip(p_depths, c_depths)):
        v = np.sqrt((xc-xp)**2 + (yc-yp)**2 + (zc-zp)**2) * 15/100
        vs.append(v)
        out_str = '3D Velocity: ' + str(v) + ' pixels/sec'
        cv2.putText(frame, out_str, (xc, yc*2-40), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 0), 1)
    
    return c_depths, vs


In [7]:
#load in the videos
cap = cv2.VideoCapture('../test-0-color.mp4')
depth_cap = cv2.VideoCapture('../test-0-depth.mp4')

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

fCount = 0

prev_faces = []
prev_depth = []

s_velos = []
c_velos = []

odd = True
depth_value = 0

'''
Gather cumulative depths/speed in 2d and 3d
'''

if (cap.isOpened()== False):
    print("error opening")

odd = False
dret = None
p_depth = []
dframe = None
while(cap.isOpened()):
    ret, frame = cap.read()
    
    if odd:
        dret, dframe = depth_cap.read()
        odd = False
    else:
        odd = True
    
    if ret:
        faces = face_cascade.detectMultiScale(frame, 1.1, 4)
        
        s_velos = s_velos + calcSpeed2D(prev_faces, faces, 4, frame)
        
        p_depth, c_c_velos = calcSpeed3D(frame, dframe, p_depth, faces, odd)
        c_velos += c_c_velos
        
        # show frame
        cv2.imshow('frame', frame)
        prev_faces = faces

        fCount += 1
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()

# look at stats for calcs
d = np.subtract(s_velos, c_velos)

# compute the average difference of sample
print("Stats for 2D speed model")
print("Mean: "  + str(np.mean(s_velos)))
print("Std Dev: " + str(np.std(s_velos)))
print("Min: " + str(np.min(s_velos)))
print("10p: " + str(np.percentile(s_velos, 10)))
print("25p: " + str(np.percentile(s_velos, 25)))
print("Med: " + str(np.percentile(s_velos, 50)))
print("75p: " + str(np.percentile(s_velos, 75)))
print("Max: " + str(np.max(s_velos)))
print("------------------------")
print("Stats for 3D speed model")
print("Mean: "  + str(np.mean(c_velos)))
print("Std Dev: " + str(np.std(c_velos)))
print("Min: " + str(np.min(c_velos)))
print("10p: " + str(np.percentile(c_velos, 10)))
print("25p: " + str(np.percentile(c_velos, 25)))
print("Med: " + str(np.percentile(c_velos, 50)))
print("75p: " + str(np.percentile(c_velos, 75)))
print("Max: " + str(np.max(c_velos)))

print("------------------------")
print("Average Difference (2D minus 3D)")
avg_diff = np.mean(d)
print(avg_diff)


Stats for 2D speed model
Mean: 167.5
Std Dev: 153.51877639520956
Min: 0.0
10p: 15.0
25p: 30.0
Med: 105.0
75p: 270.0
Max: 630.0
------------------------
Stats for 3D speed model
Mean: 173.09500915401216
Std Dev: 200.57035001707615
Min: 0.0
10p: 3.0570923678472157
25p: 36.46519639794734
Med: 80.18205640633073
75p: 231.18789150519413
Max: 839.250026809651
------------------------
Average Difference (2D minus 3D)
-5.5950091540121765


In [8]:
###
# Capture RGB and Depth data to file
###
import pyrealsense2 as rs

pipeline = rs.pipeline()
config = rs.config()


config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

# If we don't set this to 15 fps we get an error
config.enable_stream(rs.stream.depth, 640, 360, rs.format.z16, 15)

cc = cv2.VideoWriter('MP4V')
rgb_out = cv2.VideoWriter('sample-color.mp4', cc, 30, (1280, 720))
dep_out = cv2.VideoWriter('sample-depth.mp4', cc, 15, (640,  360))

while(True):   
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    depth_frame = frames.get_depth_frame()
    
        # Should be every time
    if color_frame:
        color_frame = cv2.cvtColor(color_frame, cv2.COLOR_BGR2RGB)
        rgb_out.write(color_frame)
    
    if depth_frame:
        # Update depth frame to read 16u_1 and write 8u_3 where chan 3 is 0
        x, y, z = depth_frame.shape
        d_frame = np.zeros((x, y, 3))
        for i in range(x):
            for j in range(y):
                d_frame[i][j][0] = depth_frame[i][j] & 0xff00
                d_frame[i][j][1] = depth_frame[i][j] & 0x00ff
        dep_out.write(d_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
pipeline.stop()